# Building the Sedimentation Tank Unit Model
## Part 2: The Settling Unit


Unit models encapsulate the equations, variables, and parameters that describe a phyiscal unit operation. In the image below, you can see a simple depiction of the inputs and outputs for the sedimentation unit model we intend to develop.

```
     ┌─────────────────────┐
     │                     │ → OVERFLOW (clear water)
IN → │  SEDIMENTATION TANK │
     │                     │ ↓ UNDERFLOW (sludge)
     └─────────────────────┘
```

**Model Description (typical primary clarifier):**
- 99% of water exits as overflow (clarified effluent)
- 70% of TSS settles to underflow (sludge)
- Temperature stays the same (isothermal)
- Pressure stays the same (isobaric)

---
In this tutorial, you will:
- Understand the basic structure of WaterTAP unit models
- Build a working unit model

**Goal**: Create a unit model of a sedimentation tank that separates solids from water

**Time**: 10-15 minutes

## Step 1: Import Modules

In [1]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    NonNegativeReals,
    units as pyunits,
)
from pyomo.common.config import ConfigBlock, ConfigValue, In

from idaes.core import (
    declare_process_block_class,
    UnitModelBlockData,
    FlowsheetBlock,
    useDefault,
)
from idaes.core.util.config import is_physical_parameter_block

## Step 2: Define the Unit Model Class

Fill in the **TWO TODO** sections to complete the mass balance!

<details>
  <summary>Click the arrow for hint #1!</summary>
    
The equation for water recovery is: `overflow water mass flow = water_recovery * inlet water mass flow`.
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
The syntax for defining mass flow is: `b.Port[0].flow_mass_phase_comp["Liq", "Component"]`
</details>

<details>
  <summary>Click the arrow for hint #3!</summary>
    
The equation for solids removal is: `underflow TSS mass flow = solids_removal * inlet TSS mass flow`.
</details>



In [2]:
@declare_process_block_class("SedimentationTank")
class SedimentationTankData(UnitModelBlockData):
    """
    Simple sedimentation tank unit model
    """
    
    # Configuration options
    CONFIG = ConfigBlock()
    CONFIG.declare(
        "dynamic",
        ConfigValue(default=False, domain=In([False])),
    )
    CONFIG.declare(
        "has_holdup",
        ConfigValue(default=False, domain=In([False])),
    )
    CONFIG.declare(
        "property_package",
        ConfigValue(default=useDefault, domain=is_physical_parameter_block),
    )
    CONFIG.declare(
        "property_package_args",
        ConfigBlock(implicit=True),
    )
    
    def build(self):
        super().build()
        
        # ============================================
        # Unit Model Variables
        # ============================================
        
        # What fraction of water goes to overflow?
        self.water_recovery = Var(
            initialize=0.99,
            bounds=(0, 1),
            units=pyunits.dimensionless,
            doc="Fraction of water that exits as overflow (clarified water)",
        )
        
        # What fraction of solids settles?
        self.solids_removal = Var(
            initialize=0.70,
            bounds=(0, 1),
            units=pyunits.dimensionless,
            doc="Fraction of TSS that settles to underflow (sludge)",
        )

        # ============================================
        # Ports (inlets/outlets)
        # ============================================

        # Add state blocks for inlet, outlet, and waste
        # These include the state variables and any other properties on demand
        # Add inlet block
        tmp_dict = dict(**self.config.property_package_args)
        tmp_dict["has_phase_equilibrium"] = False
        tmp_dict["parameters"] = self.config.property_package
        tmp_dict["defined_state"] = True  # inlet block is an inlet
        self.properties_in = self.config.property_package.state_block_class(
            self.flowsheet().config.time, doc="Material properties of inlet", **tmp_dict
        )
        # Add outlet and waste block
        tmp_dict["defined_state"] = False  # outlet and waste block is not an inlet
        self.properties_overflow = self.config.property_package.state_block_class(
            self.flowsheet().config.time,
            doc="Material properties of overflow",
            **tmp_dict,
        )
        self.properties_underflow = self.config.property_package.state_block_class(
            self.flowsheet().config.time, doc="Material properties of underflow", **tmp_dict
        )
        
        # Add ports - oftentimes users interact with these rather than the state blocks
        self.add_inlet_port(name="inlet", block=self.properties_in)
        self.add_outlet_port(name="overflow", block=self.properties_overflow)
        self.add_outlet_port(name="underflow", block=self.properties_underflow)
        
        # ============================================
        # Constraints 
        # ============================================
        
        # Mass balance: what comes in must go out
        @self.Constraint(
            self.config.property_package.component_list,
            doc="Mass balance"
        )
        def mass_balance(b, j):
            return (
                b.properties_in[0].flow_mass_phase_comp["Liq", j]
                == b.properties_overflow[0].flow_mass_phase_comp["Liq", j]
                + b.properties_underflow[0].flow_mass_phase_comp["Liq", j]
            )
        
        # Water recovery equation
        @self.Constraint(doc="Water recovery to overflow")
        def water_recovery_eq(b):
            # ============================================
            # TODO 1: Define water recovery
            # ============================================

            return (
                b.properties_overflow[0].flow_mass_phase_comp["Liq", "H2O"]
                == b.water_recovery * b.properties_in[0].flow_mass_phase_comp["Liq", "H2O"]
            )
        
        # Solids removal equation
        @self.Constraint(doc="Solids removal to underflow")
        def solids_removal_eq(b):
            # ============================================
            # TODO 2: Define solids removal
            # ============================================

            
            return (
                b.properties_underflow[0].flow_mass_phase_comp["Liq", "TSS"]
                == b.solids_removal * b.properties_in[0].flow_mass_phase_comp["Liq", "TSS"]
            )
        
        # Temperature stays constant
        @self.Constraint(doc="Isothermal overflow")
        def isothermal_overflow(b):
            return b.properties_in[0].temperature == b.properties_overflow[0].temperature
        
        @self.Constraint(doc="Isothermal underflow")
        def isothermal_underflow(b):
            return b.properties_in[0].temperature == b.properties_underflow[0].temperature
        
        # Pressure stays constant
        @self.Constraint(doc="Isobaric overflow")
        def isobaric_overflow(b):
            return b.properties_in[0].pressure == b.properties_overflow[0].pressure
        
        @self.Constraint(doc="Isobaric underflow")
        def isobaric_underflow(b):
            return b.properties_in[0].pressure == b.properties_underflow[0].pressure

## Step 3: Test the model

Let's test our model by simulating the sedimentation process

In [3]:
from watertap.core.solvers import get_solver

# Run the property model notebook first
%run sedimentation_property_model.ipynb

# Create model
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Add property package
m.fs.properties = SedimentationParameterBlock()

# Add sedimentation tank unit
m.fs.settler = SedimentationTank(property_package=m.fs.properties)

# Set inlet conditions (primary clarifier influent)
m.fs.settler.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.99)  # 0.99 kg/s water
m.fs.settler.properties_in[0].flow_mass_phase_comp["Liq", "TSS"].fix(0.01)  # 0.01 kg/s TSS (~250 mg/L)
m.fs.settler.properties_in[0].temperature.fix(273.15 + 20)  # 20°C
m.fs.settler.properties_in[0].pressure.fix(101325)  # Atmospheric

# Fix settler performance (typical primary clarifier)
m.fs.settler.water_recovery.fix(0.99)  # 99% water to overflow
m.fs.settler.solids_removal.fix(0.70)  # 70% TSS settles

# Solve!
solver = get_solver()
results = solver.solve(m)

print("\n" + "="*60)
print("PRIMARY CLARIFIER SIMULATION COMPLETE!")
print("="*60)

# Calculate some useful metrics
inlet_tss_conc = (m.fs.settler.properties_in[0].flow_mass_phase_comp['Liq', 'TSS'].value / 
                  (m.fs.settler.properties_in[0].flow_mass_phase_comp['Liq', 'H2O'].value + 
                   m.fs.settler.properties_in[0].flow_mass_phase_comp['Liq', 'TSS'].value)) * 1e6

overflow_tss_conc = (m.fs.settler.properties_overflow[0].flow_mass_phase_comp['Liq', 'TSS'].value / 
                     (m.fs.settler.properties_overflow[0].flow_mass_phase_comp['Liq', 'H2O'].value + 
                      m.fs.settler.properties_overflow[0].flow_mass_phase_comp['Liq', 'TSS'].value)) * 1e6

print("\n INLET (Primary Influent):")
print(f"   Water:     {m.fs.settler.properties_in[0].flow_mass_phase_comp['Liq', 'H2O'].value:.3f} kg/s")
print(f"   TSS:       {m.fs.settler.properties_in[0].flow_mass_phase_comp['Liq', 'TSS'].value:.3f} kg/s")
print(f"   TSS conc:  {inlet_tss_conc:.1f} mg/L")
print(f"   Temp:      {m.fs.settler.properties_in[0].temperature.value - 273.15:.1f}°C")

print("\n OVERFLOW (Clarified Effluent):")
print(f"   Water:     {m.fs.settler.properties_overflow[0].flow_mass_phase_comp['Liq', 'H2O'].value:.3f} kg/s")
print(f"   TSS:       {m.fs.settler.properties_overflow[0].flow_mass_phase_comp['Liq', 'TSS'].value:.4f} kg/s")
print(f"   TSS conc:  {overflow_tss_conc:.1f} mg/L")
print(f"   Temp:      {m.fs.settler.properties_overflow[0].temperature.value - 273.15:.1f}°C")

print("\n  UNDERFLOW (Settled Sludge):")
print(f"   Water:     {m.fs.settler.properties_underflow[0].flow_mass_phase_comp['Liq', 'H2O'].value:.4f} kg/s")
print(f"   TSS:       {m.fs.settler.properties_underflow[0].flow_mass_phase_comp['Liq', 'TSS'].value:.4f} kg/s")
print(f"   Temp:      {m.fs.settler.properties_underflow[0].temperature.value - 273.15:.1f}°C")

tss_removal_pct = (1 - overflow_tss_conc/inlet_tss_conc) * 100
print("\n PERFORMANCE:")
print(f"   TSS Removal: {tss_removal_pct:.1f}%")
print(f"   Water Recovery: {m.fs.settler.water_recovery.value*100:.1f}%")

print("\n" + "="*60)
print(" Mass balance satisfied!")
print("="*60)


SUCCESS! Your sedimentation property model works!

 Wastewater Stream Properties:
   Water flow:     0.990 kg/s
   TSS flow:       0.010 kg/s
   Temperature:    20.0°C
   Pressure:       101325 Pa

 Calculated Properties:
   Water fraction: 50.0%
   TSS fraction:   50.0%
   Flow rate:      86.4 m³/day

PRIMARY CLARIFIER SIMULATION COMPLETE!

 INLET (Primary Influent):
   Water:     0.990 kg/s
   TSS:       0.010 kg/s
   TSS conc:  10000.0 mg/L
   Temp:      20.0°C

 OVERFLOW (Clarified Effluent):
   Water:     0.980 kg/s
   TSS:       0.0030 kg/s
   TSS conc:  3051.6 mg/L
   Temp:      20.0°C

  UNDERFLOW (Settled Sludge):
   Water:     0.0099 kg/s
   TSS:       0.0070 kg/s
   Temp:      20.0°C

 PERFORMANCE:
   TSS Removal: 69.5%
   Water Recovery: 99.0%

 Mass balance satisfied!
 Your unit model works perfectly!


## Summary

In this tutorial, we learned how to create a working unit model by: 
- Creating stateblocks for inlet, overflow, and underflow streams  
- Writing mass balances, removal equations, and other constraints  

**This structure is IDENTICAL for EVERY WaterTAP unit model!**

Every WaterTAP unit model has:

1. **Variables** (water_recovery, solids_removal)
2. **Ports** (inlet, overflow, underflow)
3. **Constraints** (mass balance, removal equations)

---

## Compare to Week 4

Open the Week 4 filtration model and look at it now.

**You'll see that the core structure is identical to what you just built:**
- Same variable types (Var, Constraint)
- Same state block pattern (inlet/outlets)
- Same mass balance approach
- Just more components and more complexity

---

## Week 4 Takeaway

Week 4 wasn't about memorizing code.  
It was about recognizing the following pattern:

```
1. Define components and parameters (Parameter Block - Property Model)
   ↓
2. Create state variables (State Block - Property Model)
   ↓
3. Add unit variables (Unit Model)
   ↓
4. Write constraints (Unit Model)
   ↓
5. Solve!
```

**Now that you understand this workflow, you'll have an easier time
deciphering the existing code, modifying it for your own purposes,
and/or creating your own models.**
